In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [2]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

from tensorflow.keras import layers
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import csv
import scipy.stats as stats
from tensorboard.plugins.hparams import api as hp
import scipy.stats as stats
plt.rcParams["figure.figsize"] = (20,10)

import seaborn as sns
from Data import Data
from datetime import datetime
import feature_finder_keiko as ff


/home/mathsys2/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Could not import rpy package
Could not import r-package RCIT


In [3]:
tau = 6

n_components_sst = 98

link_caus = np.load("./link_rotated_station/best_link_1976_2010_{}.npy".format(tau))
link_corr = np.load("./link_rotated_station/best_link_corr_1976_2010_{}.npy".format(tau))


train_start = 1976
validation_end = 2010

temporal_limits = {"time_min":datetime(train_start, 1, 1, 0, 0),"time_max":datetime(validation_end, 12, 1, 0, 0)}
original_count, count = ff.drought_timeseries("../npy_files/ET_gamma_18912015.npy",train_start,validation_end)
#data_sst, ts, V, df_sst, avg, std = ff.PCA_computer_rotated(file_name, "sst",temporal_limits, n_components_sst, -9.96921e+36)

data_sst = np.load("data_sst.npy")
V = np.load("V.npy")
ts = np.load("ts.npy")
avg = np.load("avg.npy")
std = np.load("std.npy")
data_sst_test = np.load("data_sst_test.npy")

df_sst = pd.read_csv("df_sst.csv")

test_start = 2011
test_end = 2015

temporal_limits_test = {"time_min":datetime(test_start, 1, 1, 0, 0),"time_max":datetime(test_end, 12, 1, 0, 0)}
#data_sst_test = ff.data_generator_avg_std('../../../nc/sst.mnmean.nc', "sst",temporal_limits_test, avg, std, 12, -9.96921e+36)
original_count_test, count_test = ff.drought_timeseries("../npy_files/ET_gamma_18912015.npy",test_start,test_end)

del df_sst["Unnamed: 0"]

In [4]:
#Hybrid

for i in range(len(link_corr)):
    if not np.array(link_corr == list(link_corr[i])).all(axis=1).all():
        link_caus = np.concatenate((link_caus, np.reshape(link_corr[i,:],(1,2))))
link_hybrid = link_caus
link_caus = np.load("./link_rotated_station/best_link_1976_2010_{}.npy".format(tau))

l, b, m = ff.forward_feature_V(count, data_sst, link_hybrid, V, tau)

link = l

start_lag = tau
end_lag = tau + 12

df_test = pd.DataFrame({"drought":original_count_test})
lags = np.arange(start_lag,end_lag + 1)
df_test = df_test.assign(**{
'{} (t-{})'.format(col, t): df_test[col].shift(t)
for t in lags
for col in df_test
})
for k in range(len(link)):
    df_sst["pc"] = V[:,link[k,0]-1]
    df_test[str(k)] = ff.time_series_maker(link[k,0]-1, df_sst, data_sst_test)
    df_test[str(k)] = df_test[str(k)].shift(abs(link[k,1]))
df_test = df_test.dropna()

x_test = df_test.iloc[:,1:]
y_test = df_test.iloc[:,0]

In [5]:
link = l

start_lag = tau
end_lag = tau+12

df = pd.DataFrame({"drought": original_count})
lags = np.arange(start_lag,end_lag + 1)
df = df.assign(**{
'{} (t-{})'.format(col, t): df[col].shift(t)
for t in lags
for col in df
})
for k in range(len(link)):
    df[str(k)] = ff.time_series_maker_V(data_sst, V[:,link[k,0]-1])
    df[str(k)] = df[str(k)].shift(abs(link[k,1]))
df = df.dropna()

x_train = df.iloc[:,1:]
y_train = df.iloc[:,0]

start_lag = tau
end_lag = tau + 12

df_test = pd.DataFrame({"drought":original_count_test})
lags = np.arange(start_lag,end_lag + 1)
df_test = df_test.assign(**{
'{} (t-{})'.format(col, t): df_test[col].shift(t)
for t in lags
for col in df_test
})
for k in range(len(link)):
    df_sst["pc"] = V[:,link[k,0]-1]
    df_test[str(k)] = ff.time_series_maker(link[k,0]-1, df_sst, data_sst_test)
    df_test[str(k)] = df_test[str(k)].shift(abs(link[k,1]))
df_test = df_test.dropna()

x_test = df_test.iloc[:,1:]
y_test = df_test.iloc[:,0]

In [39]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

#model = tf.keras.Sequential([
#tf.keras.layers.InputLayer(input_shape = x_train.shape[1]),
#layers.Dense(16, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation=tf.nn.relu),
#layers.Dropout(0.1),
#layers.Dense(16, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation=tf.nn.relu),
#layers.Dropout(0.1),
#layers.Dense(16, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation=tf.nn.relu),
#layers.Dense(1)
#])

model = tf.keras.Sequential([
tf.keras.layers.InputLayer(input_shape = x_train.shape[1]),
layers.Dense(16, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation=tf.nn.relu),
layers.Dropout(0.1),
layers.Dense(16, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation=tf.nn.relu),
layers.Dropout(0.1),
layers.Dense(16, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation=tf.nn.relu),
layers.Dropout(0.1),
layers.Dense(8, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation=tf.nn.relu),
layers.Dense(1)
])



#model = keras.models.Sequential([
#    keras.layers.Dense(16, input_dim=1),
#    keras.layers.Dense(1),
#])

model.compile(
    loss='mean_squared_error', # keras.losses.mean_squared_error
    optimizer=keras.optimizers.RMSprop(lr=1e-06),
    metrics=['RootMeanSquaredError']
)

print("Training ... With default parameters, this takes less than 10 seconds.")
training_history = model.fit(
    x_train, # input
    y_train, # output
    verbose=True, # Suppress chatty output; use Tensorboard instead
    epochs=100,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback],
)

print("Average test loss: ", np.average(training_history.history['loss']))


Training ... With default parameters, this takes less than 10 seconds.
Train on 402 samples, validate on 42 samples
Epoch 1/100
402/402 [==============================] - 1s 2ms/sample - loss: 3639.1785 - RootMeanSquaredError: 60.3206 - val_loss: 7808.4616 - val_RootMeanSquaredError: 88.3621
Epoch 2/100
402/402 [==============================] - 0s 65us/sample - loss: 3614.8324 - RootMeanSquaredError: 60.1184 - val_loss: 7819.8491 - val_RootMeanSquaredError: 88.4265
Epoch 3/100
402/402 [==============================] - 0s 71us/sample - loss: 3620.7007 - RootMeanSquaredError: 60.1672 - val_loss: 7832.5502 - val_RootMeanSquaredError: 88.4983
Epoch 4/100
402/402 [==============================] - 0s 77us/sample - loss: 3598.9044 - RootMeanSquaredError: 59.9858 - val_loss: 7843.3683 - val_RootMeanSquaredError: 88.5594
Epoch 5/100
402/402 [==============================] - 0s 80us/sample - loss: 3624.0720 - RootMeanSquaredError: 60.1952 - val_loss: 7855.7121 - val_RootMeanSquaredError: 88.

402/402 [==============================] - 0s 63us/sample - loss: 3584.1591 - RootMeanSquaredError: 59.8628 - val_loss: 8338.8617 - val_RootMeanSquaredError: 91.3140
Epoch 47/100
402/402 [==============================] - 0s 67us/sample - loss: 3550.0751 - RootMeanSquaredError: 59.5774 - val_loss: 8351.4808 - val_RootMeanSquaredError: 91.3831
Epoch 48/100
402/402 [==============================] - 0s 65us/sample - loss: 3662.6574 - RootMeanSquaredError: 60.5149 - val_loss: 8363.1483 - val_RootMeanSquaredError: 91.4469
Epoch 49/100
402/402 [==============================] - 0s 68us/sample - loss: 3477.9887 - RootMeanSquaredError: 58.9693 - val_loss: 8376.8098 - val_RootMeanSquaredError: 91.5216
Epoch 50/100
402/402 [==============================] - 0s 64us/sample - loss: 3633.8624 - RootMeanSquaredError: 60.2765 - val_loss: 8387.9041 - val_RootMeanSquaredError: 91.5822
Epoch 51/100
402/402 [==============================] - 0s 65us/sample - loss: 3507.9987 - RootMeanSquaredError: 59.22

Epoch 92/100
402/402 [==============================] - 0s 66us/sample - loss: 3474.4824 - RootMeanSquaredError: 58.9396 - val_loss: 8920.6370 - val_RootMeanSquaredError: 94.4459
Epoch 93/100
402/402 [==============================] - 0s 65us/sample - loss: 3458.1369 - RootMeanSquaredError: 58.8008 - val_loss: 8933.7665 - val_RootMeanSquaredError: 94.5154
Epoch 94/100
402/402 [==============================] - 0s 65us/sample - loss: 3498.1848 - RootMeanSquaredError: 59.1403 - val_loss: 8947.5196 - val_RootMeanSquaredError: 94.5881
Epoch 95/100
402/402 [==============================] - 0s 80us/sample - loss: 3485.1365 - RootMeanSquaredError: 59.0299 - val_loss: 8959.2959 - val_RootMeanSquaredError: 94.6504
Epoch 96/100
402/402 [==============================] - 0s 83us/sample - loss: 3537.5163 - RootMeanSquaredError: 59.4719 - val_loss: 8972.3493 - val_RootMeanSquaredError: 94.7193
Epoch 97/100
402/402 [==============================] - 0s 75us/sample - loss: 3425.8238 - RootMeanSquare

In [8]:
%tensorboard --logdir logs/scalars